In [ ]:
using Random
using Statistics

using AbstractPlotting
using CairoMakie
using JuMP
using SCIP
using Triangle

In [ ]:
Random.seed!(1);

In [ ]:
n = 8

x = 400 * rand(n)
y = 200 * rand(n)
[x y]'

In [ ]:
make_scene(width=400, height=200) = Scene(resolution=(width, height), show_axis=false, scale_plot=false)

In [ ]:
function unique_edges(triangles)
    set = Set()
    for t in triangles
        push!(set, min(t[[1, 2]], t[[2, 1]]))
        push!(set, min(t[[2, 3]], t[[3, 2]]))
        push!(set, min(t[[1, 3]], t[[3, 1]]))
    end
    return hcat(sort(collect(set))...)
end

In [ ]:
# Start with straight-forward Delaunay triangulation.
points = [x y]
points_map = collect(1:n)

triangles = basic_triangulation(points, points_map)
@show triangles

edges = unique_edges(triangles)
@show edges

make_scene()
scatter!(x, y, markersize=8, color=:red)
linesegments!(points[vec(edges), :])

In [ ]:
# for each triangle, add a point in the center (mean)
centers = [mean(points[t, :], dims=1) for t in triangles]
centers_array = reshape(hcat(centers...), 2, :)'

In [ ]:
# triangulate with added center points, keeping existing edges
points2 = vcat(points, centers_array)
points_map2 = collect(1:size(points2, 1))
triangles2 = constrained_triangulation(points2, points_map2, edges'[:,:])

edges2 = unique_edges(triangles2)
make_scene()
linesegments!(points[vec(edges), :])
linesegments!(points2[vec(edges2), :], color=:green)
scatter!(centers_array, markersize=4)
scatter!(x, y, markersize=5, color=:red)

In [ ]:
# compare triangulation without keeping edges
points3 = vcat(points, centers_array)
points_map3 = collect(1:size(points3, 1))
triangles3 = basic_triangulation(points3, points_map3)
@show triangles3

edges3 = unique_edges(triangles3)
@show edges3

make_scene()
linesegments!(points3[vec(edges3), :], color=:green)
scatter!(x, y, markersize=5, color=:red)
scatter!(centers_array)

In [ ]:
# find edge centers for subdivision
edge_centers = mean(points[edges, :], dims=1)[1, :, :]

# triangulate with added center points and edge subdivision, keeping old edges
points4 = vcat(points, centers_array, edge_centers)
points_map4 = collect(1:size(points4, 1))
triangles4 = constrained_triangulation(points4, points_map4, edges'[:,:])
edges4 = unique_edges(triangles4)

make_scene()
linesegments!(points4[vec(edges4), :], color=:orange)
linesegments!(points[vec(edges), :])
scatter!(edge_centers, markersize=4, color=:green)
scatter!(centers_array, markersize=4, color=:cyan)
scatter!(x, y, markersize=5, color=:red)

In [ ]:
function subdivided_edges(edges, offset)
    set = Vector()
    for e in 1:size(edges, 2)
        edge = edges[:, e]
        push!(set, [edge[1], e + offset])
        push!(set, [e + offset, edge[2]])
    end
    return hcat(set...)
end

In [ ]:
# triangulate with added center points and edge subdivision, keeping subdivided edges
points5 = vcat(points, centers_array, edge_centers)
points_map5 = collect(1:size(points5, 1))

offset = size(points, 1) + size(centers_array, 1)
sub_edges = subdivided_edges(edges, offset)

triangles5 = constrained_triangulation(points5, points_map5, sub_edges'[:,:])
edges5 = unique_edges(triangles5)

make_scene()
linesegments!(points4[vec(edges5), :], color=:orange)
linesegments!(points[vec(edges), :])
scatter!(edge_centers, markersize=4, color=:green)
scatter!(centers_array, markersize=4, color=:cyan)
scatter!(x, y, markersize=5, color=:red)